In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import statistics as stat

In [2]:
data = pd.read_csv('scdb-test-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

5834

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

accept_count = 0
accept_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'SELL':
        accept_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        accept_count += 1   


print(f"#SELLs =  {accept_count}")

#SELLs =  89


In [5]:
time_all_committed_accepts = {}
time_passed_accepts = []
validated_time = []
accepted_time = []
committed_time = []

# accepts with more than 6 logs
accept_underfit = []
accept_underfit_dict = {}

accept_overfit = []
accept_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "SELL" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            accept_underfit.append(temp_list)
            accept_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            accept_overfit.append(actual_time)
            accept_overfit_dict[tx_id[i]] = temp_list

In [6]:
len(time_passed_accepts)

5

In [7]:
len(accept_overfit)

0

In [8]:
len(accept_underfit)

82

In [9]:
accept_underfit_dict

{'024b1f603bd74d79a643504d5c35870466322fbfaf97867bba7c0635451b9e4d': [48],
 '032e8977bbb7d782b0e5a689af0d0c3afc32fd1a9cf9f869e79e5242749b65e6': [117],
 '07f9e0c5d7e9b5781bf0a7981dbc3f17a930182c7ca5b3cbd62cb904a5810fba': [47],
 '08a1203b0500e33916e7a53828727429332c7d442c71fd42f24feca7a280557b': [7646,
  7652],
 '0fabff7fa8c966d9e7fccd1e655dc4e9ee80c1ae30eff59811c7e06008cf05f9': [48],
 '10b2528cace42e86d2f11bbe9da0df5f0994d92ddb8bb61633d0fccdb2a457d8': [56],
 '10f5606a56aefd775106df2c85d2819866a3bdbd8c8fd1b1100214f9a488769e': [17116],
 '12a8ee71860a65348550771af15fc5ca4d969a0836c1435e1cad77ee4d68e551': [12355],
 '16d4b10d025ad6d7b69467409b6996a8a320e731318d9e28bf3234a81b8bf91f': [14660],
 '189b8da60d2c5996a96f2a3ccc8aad089b03a12b74761644adcf4bd92b10764a': [12020],
 '1b351d57dfe1906dcb18287c78bb3ce5edd7a3ecbcc8bb1f784c7721d4b53702': [43],
 '1d55606aec9e7d7190b499569e5e0a734bd0a6eecc86378abf6a62d6c392b6b3': [48],
 '1db20ff8f2c2eca4d558fa8931970dee3cc84f84e66e35864a0649e00e2edbcd': [102],
 

In [10]:
stat.mean(time_passed_accepts)

333.6

In [11]:
# Analysis of CREATE Transaction
accept_mean = "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total sells = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'average': accept_mean,
               'median' : accept_median,
               'standard_deviation' : accept_std_dev,
               '# of txs' : len(time_passed_accepts)}

333.60
283
468
225
104.14
total sells = 5


In [12]:
results = pd.DataFrame(accept_dict, index = ['SELL'])
results.to_csv('sell_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
SELL,225,468,333.60,283,104.14,5
